In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import sys
if '/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/' not in sys.path:
  sys.path.append('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/')

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
!pip install -q SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 31.6 MB/s eta 0:00:00


In [ ]:
import SimpleITK as sitk
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib import cm
from PIL import Image
import cv2

i=0 # generate the counter

for path, subdirs, files in os.walk('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/'): # generates the file names in a directory tree by  
                                                                                                       # walking through the tree either top-down or bottom-up 
    for name in files:
      if name.endswith('adc.mha'):             # select only the files that end with 'adc.mha' 
        image_path = os.path.join(path, name)  # save the absolute path of each selected file to the variable image_path.
        image = sitk.ReadImage(image_path) # reads in the selected image and saves it to the variable image.
        max_index = image.GetDepth() # gets the maximum depth of the image along its z-axis
        
        list_of_2D_images_np = [sitk.GetArrayFromImage(image[:,:,i]) for i in range(max_index)] # creates a list of 2D numpy arrays, where each element 
                                                                                                # of the list is a 2D slice of the original 3D image.
        for j in range(max_index):  # This loop iterates over all the 2D slices in list_of_2D_images_np 
                                    # and saves each slice as a 2D .png file
          plt.imshow(list_of_2D_images_np[j], cmap=cm.Greys_r) # displays the 2D image slice as a grayscale image
          plt.axis('off')
          my_dpi=120 # sets the dots-per-inch resolution of the saved .png file to 120
          save_path = image_path.replace('.mha', str(j)+'_adc.png') # constructs the file name for the saved .png file by replacing the '.mha'
                                                                    # extension of the original file with '_adc.png' and appending the slice index j as a string
          plt.savefig(save_path, bbox_inches='tight',  pad_inches = 0) # bbox_inches='tight' trims any whitespace around the edges of the plot
                                                                       # pad_inches=0 removes any padding between the plot and the edge of the saved .png file.
          #plt.show()
          plt.close() # closes the current plot

          image = Image.open(save_path)                 # open the saved .png file
          resized_image = image.resize((384,384))       # resizes it to 384x384 pixels
          resized_image = resized_image.save(save_path) # saves the resized image to the same file

          if i%50==0:                                  # check if the current image being processed is a multiple of 50 (i.e. if i is evenly divisible by 50). 
            print('Image number:', i, ' ', image_path) # If it is, the code prints a message indicating the total number of saved images and the path of 
          i+=1                                         # the currently processed file. The counter variable i is then incremented by 1.

Overall, this code reads in a set of medical images in the .mha format, extracts 2D slices from them, saves each slice as a 2D .png file, resizes the saved .png files to 384x384 pixels, and prints out an update message every 50 images processed.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import sklearn.datasets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import os

In [ ]:
path_data_t2w = '/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI' 

In [ ]:
cont=1                                                                                               
with open('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/t2w_list.txt','w+') as f:  # opens a new file called 't2w_list.txt'
                                                                                                    # for writing ('w+') and saves the file object to the variable f.                                                   
  for path, subdirs, files in os.walk(path_data_t2w): 
    for name in files:
      if name.endswith('.png'): 
        file_path = os.path.join(path, name) # saves the absolute path of the file to the text file 't2w_list.txt' 
        f.write(file_path+'\n')              # using the write() method of the file object f
        cont+=1
        if cont%100 == 0:
          print("I have printed files: ", cont)

This code walks through a directory of .png files, saves the absolute path of each file to a text file, and prints a message every 100 files processed.
Overall, this code is useful for creating a list of file paths that can be used as input to a machine learning algorithm or other analysis tool. By writing the file paths to a text file, the code makes it easy to load and process all the images in the directory without having to manually specify each file path.

In [ ]:
path_data_adc = '/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI'

In [ ]:
cont=1
with open('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/adc_list.txt','w+') as f:
  for path, subdirs, files in os.walk(path_data_adc):
    for name in files:
      if name.endswith('adc.png'):
        file_path = os.path.join(path, name)
        f.write(file_path+'\n')
        cont+=1
        if cont%100 == 0:
          print("I have printed files: ", cont)

This code walks through a directory of .png files with names ending in 'adc.png', saves the absolute path of each file to a text file called 'adc_list.txt', and prints a message every 100 files processed.
This code is similar to the previous code for creating a list of file paths, but it specifically selects files with names ending in 'adc.png'. This suggests that the images being processed in this code have already been preprocessed and saved as .png files with names that include information about the type of image (in this case, 'adc' likely refers to apparent diffusion coefficient maps used in prostate imaging). By creating separate lists of file paths for different types of images, the code makes it easy to load and process only the images of interest

In [ ]:
import os
cont=0
for path, subdirs, files in os.walk('/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/'): 
    for name in files:
      if 't2w' in name and name.endswith('.png'):
        image_path = os.path.join(path, name)
        size = os.stat(image_path).st_size/1024 # os.stat() calculates the size of the file in kilobytes
                                                # st_size attribute of the resulting object contains the size of the file in bytes
                                                # size is then divided by 1024 to convert it to kilobytes and stored in the variable size
        if 85 < size < 95: # checks if kb size is between 85 and 95 
          print(image_path)
          cont+=1
print(cont)

/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10360/10360_1000366_t2w20.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10211/10211_1000215_t2w9.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10211/10211_1000215_t2w10.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10211/10211_1000215_t2w12.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10211/10211_1000215_t2w11.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10211/10211_1000215_t2w14.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10211/10211_1000215_t2w17.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10211/10211_1000215_t2w18.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10209/10209_1000213_t2w3.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10201/10201_1000205_t2w25.png
/content/gdrive/My Drive/Colab Notebooks/ABELE_prostate/PICAI/10201/10201_1000205_

This code walks through a directory of .png files with names that contain the substring 't2w' and prints the absolute path of each file whose size is between 85 and 95 kilobytes. At the end, it prints the total number of files that met this criteria. this code is useful for selecting a subset of images that meet specific criteria, such as a specific size range. This can be useful for filtering out images that are too small or too large